In [1]:
import cv2
import numpy as np
import os
from random import shuffle
from tqdm import tqdm
IMAGE_DIR = '.\dataset\single_image_test'
IMG_SIZE = 50
LR = 1e-3
MODEL_NAME = 'socialmedia-4_27-0.001-6conv-basic-video.model'

In [2]:
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression

import tensorflow as tf
with tf.device('/gpu:0'):
    sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
    #sess = tf.Session()
    tf.reset_default_graph()

    convnet = input_data(shape=[None, IMG_SIZE, IMG_SIZE, 1], name='input')
    convnet = conv_2d(convnet, 32, 2, activation='relu')
    convnet = max_pool_2d(convnet, 2)
    convnet = conv_2d(convnet, 32, 2, activation='relu')
    convnet = max_pool_2d(convnet, 2)
    convnet = conv_2d(convnet, 32, 2, activation='relu')
    convnet = max_pool_2d(convnet, 2)
    convnet = conv_2d(convnet, 32, 2, activation='relu')
    convnet = max_pool_2d(convnet, 2)
    convnet = conv_2d(convnet, 32, 2, activation='relu')
    convnet = max_pool_2d(convnet, 2)
    convnet = conv_2d(convnet, 32, 2, activation='relu')
    convnet = max_pool_2d(convnet, 2)
    convnet = fully_connected(convnet, 1024, activation='relu')
    convnet = dropout(convnet, 0.8)
    convnet = fully_connected(convnet, 2, activation='softmax')
    convnet = regression(convnet, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets')
    model = tflearn.DNN(convnet, tensorboard_dir='log')

curses is not supported on this machine (please install/reinstall curses for an optimal experience)
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
Use tf.cast instead.


In [3]:
if os.path.exists('{}.meta'.format(MODEL_NAME)):
    model.load(MODEL_NAME)
    print('model loaded!')

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from c:\COSC_490_Project_TestEnv\socialmedia-4_27-0.001-6conv-basic-video.model
model loaded!


In [4]:
def process_test_data():
    testing_data = []
    for img in tqdm(os.listdir(IMAGE_DIR)):
        path = os.path.join(IMAGE_DIR, img)
        img_num = img.split('.')[0]
        img = cv2.resize(cv2.imread(path,cv2.IMREAD_GRAYSCALE), (IMG_SIZE,IMG_SIZE))
        testing_data.append([np.array(img), img_num])
    shuffle(testing_data)
    np.save('single_test_data.npy', testing_data)
    return testing_data

In [5]:
test_data = process_test_data()

100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 29.41it/s]


In [7]:
with open('weekly-log.csv', 'w') as f:
    f.write('id,label\n')

In [8]:
with open('weekly-log.csv', 'a') as f:
    for data in tqdm(test_data):
        img_num = data[1]
        img_data = data[0]
        orig = img_data 
        data = img_data.reshape(IMG_SIZE,IMG_SIZE,1)
        model_out = model.predict([data])[0]
        
        f.write('{},{}\n'.format(img_num, model_out[1]))
        
        ## File prints chance of image being an allowed website and not social media
        ## Any label <.5 is ok, anything >= .51 is considered social media
        ## I.E. any images from testing data with ID > 177 should have a very low label

100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 55.55it/s]
